# Limpieza de Paramount+

## Pedir datos a la API

In [1]:
# IMPORTS
import numpy as np
import pandas as pd
import requests as rq
from src.buscar_peliculas import buscar_peliculas

Pedimos los datos:

In [2]:
dfparamount_raw = buscar_peliculas('Paramount+')

<Response [200]> 2 457


Fecha de consulta en la API: 18-10-2025 22:00

Verificamos que no haya perdida de datos:

In [3]:
largo_dataframe = len(dfparamount_raw[0])
largo_segun_API = dfparamount_raw[1]
print([largo_dataframe, largo_segun_API])

[457, 457]


Los largos coinciden

In [4]:
dfparamount1 = dfparamount_raw[0]

In [5]:
dfparamount1

,id,title,year,imdb_id,tmdb_id,tmdb_type,type
0,11035730,Ozzy: No Escape from Now,2025,tt35922832,1508191,movie,movie
1,1535445,Gladiator II,2024,tt9218128,558449,movie,movie
2,1437213,Top Gun: Maverick,2022,tt1745960,361743,movie,movie
3,1856741,Better Man,2024,tt14260836,799766,movie,movie
4,1583908,The Menu,2022,tt9764362,593643,movie,movie
...,...,...,...,...,...,...,...
452,1702437,Kiss the Future,2024,tt17022868,1075845,movie,movie
453,1702733,Superpower,2023,tt18327788,943330,movie,movie
454,1623462,Sabaya,2021,tt13567374,776555,movie,movie
455,2749397,Smile,2023,tt22982202,1038522,movie,movie


Guardamos estos datos crudos en un csv

In [6]:
dfparamount1.to_csv('data/raw_api_paramount.csv', index=False)

## Limpieza
### Columnas
Las columnas nos entregan los siguientes datos:
- `id`: Identificador de la pelicula en la API
- `title`: Nombre de la pelicula
- `year`: Año de estreno de la pelicula
- `imdb_id`: Identificador de la pelicula en IMDB
- `tmdb_id`: Identificador de la pelicula en TMDB
- `tmdb_type`: Tipo del titulo en TMDB
- `type`: Tipo del titulo en la API

### Eliminar Columnas
Como le especificamos a la API que nos entregara solo peliculas, la columna `type` y `tmdb_type` no deberian entregar informacion importante, ya que todas las filas tienen el mismo tipo

In [7]:
dfparamount1['type'].unique()

array(['movie'], dtype=object)

In [8]:
dfparamount1['tmdb_type'].unique()

array(['movie'], dtype=object)

Al ser todos del mismo tipo, ambas columnas pueden ser eliminadas

In [9]:
dfparamount2 = dfparamount1[['id', 'title', 'year', 'imdb_id', 'tmdb_id']]

### Valores Na

In [10]:
resultados = {}
for i in dfparamount2.columns:
    resultados[i] = dfparamount2[i].isna().sum()
resultados

{'id': np.int64(0),
 'title': np.int64(0),
 'year': np.int64(0),
 'imdb_id': np.int64(0),
 'tmdb_id': np.int64(0)}

### Duplicados

In [11]:
dfparamount2.duplicated().sum()

np.int64(0)

#### id
Deberia ser unico para cada pelicula

In [12]:
dfparamount2.duplicated('id').sum()

np.int64(0)

#### title
Es posible que se repita, ya que hay peliculas con el mismo titulo

In [13]:
dfparamount2.duplicated('title').sum()

np.int64(9)

revisemos porque se repiten los titulos de las peliculas

In [14]:
dfparamount2[dfparamount2['title'] == 'Smile']

,id,title,year,imdb_id,tmdb_id
6,1645477,Smile,2022,tt15474916,882598
444,1698365,Smile,2022,tt15751968,1051335
455,2749397,Smile,2023,tt22982202,1038522


Son distintas peliculas con mismo nombre

#### imdb_id

In [15]:
dfparamount2.duplicated('imdb_id').sum()

np.int64(0)

### Tipos de columnas

In [17]:
dfparamount2.dtypes

id          int64
title      object
year        int64
imdb_id    object
tmdb_id     int64
dtype: object

Los tipos de las columnas estan bien

### Analisis de columna `year`

In [18]:
dfparamount2['year'].describe()

count     457.000000
mean     2007.925602
std        13.357883
min      1933.000000
25%      2000.000000
50%      2009.000000
75%      2020.000000
max      2025.000000
Name: year, dtype: float64

todos estan dentro de un rango logico

## Creacion archivo `.csv`
Como ya tenemos los datos correctos y limpios, procedemos a guardar en un archivo `.csv`

In [20]:
dfparamount2.to_csv('data/api_paramount.csv', index=False)